In [ ]:
!pip install setfit datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Load your dataset
data = pd.read_csv("/content/final_dataset.csv")
print("Original class distribution:")
print(data["class_value"].value_counts())

# Separate the data by class
class_0 = data[data["class_value"] == 0]
class_1 = data[data["class_value"] == 1]

# Downsample class 0 to match the size of class 1 (65 samples)
class_0_downsampled = class_0.sample(n=74, random_state=42)

# Concatenate the downsampled class 0 with class 1 to create a balanced dataset
balanced_data = pd.concat([class_0_downsampled, class_1], axis=0)
balanced_data = balanced_data.rename(columns={"class_value": "label", "concatenated_text": "text"})


# Shuffle the dataset to avoid ordering bias
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the new class distribution
print("\nBalanced class distribution:")
print(balanced_data["label"].value_counts())

# Split into train and test sets (80-20 split)
train_data, eval_data  = train_test_split(balanced_data, stratify=balanced_data["label"],test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_data)
eval_dataset = Dataset.from_pandas(eval_data)

print(f"Training samples: {train_data.shape}, Test samples: {eval_data.shape}")

Original class distribution:
class_value
0    114
1     74
Name: count, dtype: int64

Balanced class distribution:
label
1    74
0    74
Name: count, dtype: int64
Training samples: (118, 2), Test samples: (30, 2)


In [ ]:
train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 118
})

In [ ]:
train_dataset = train_dataset.remove_columns("__index_level_0__")
eval_dataset = eval_dataset.remove_columns("__index_level_0__")


In [ ]:
from setfit import SetFitModel, SetFitTrainer,TrainingArguments
from sentence_transformers.losses import CosineSimilarityLoss

# Load a SetFit model from Hugging Face
model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")

# Check if the model is loaded correctly
print(f"Loaded model: {model}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Loaded model: <setfit.modeling.SetFitModel object at 0x78f37a7013f0>


In [ ]:
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)

<ipython-input-6-8d780ef5006f>:1: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(


Map:   0%|          | 0/118 [00:00<?, ? examples/s]

In [ ]:
trainer.train()


***** Running training *****
  Num unique pairs = 4720
  Batch size = 16
  Num epochs = 1
/usr/local/lib/python3.10/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Load your dataset
data = pd.read_csv("/content/final_dataset.csv")
print("Original class distribution:")
print(data["class_value"].value_counts())

# Separate the data by class
class_0 = data[data["class_value"] == 0]
class_1 = data[data["class_value"] == 1]

# Downsample class 0 to match the size of class 1 (65 samples)
class_0_downsampled = class_0.sample(n=74, random_state=42)

# Identify the samples that were dropped
dropped_samples = class_0[~class_0.index.isin(class_0_downsampled.index)]

# Save the dropped samples to another CSV file
dropped_samples.to_csv("dropped_samples.csv", index=False)

# Concatenate the downsampled class 0 with class 1 to create a balanced dataset
balanced_data = pd.concat([class_0_downsampled, class_1], axis=0)
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the new class distribution
print("\nBalanced class distribution:")
print(balanced_data["class_value"].value_counts())

# Split into train and test sets (80-20 split)
train_data, eval_data  = train_test_split(balanced_data, stratify=balanced_data["class_value"],test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_data)
eval_dataset = Dataset.from_pandas(eval_data)

print(f"Training samples: {train_data.shape}, Test samples: {eval_data.shape}")

Original class distribution:
class_value
0    114
1     74
Name: count, dtype: int64

Balanced class distribution:
class_value
1    74
0    74
Name: count, dtype: int64
Training samples: (118, 2), Test samples: (30, 2)


In [ ]:
from setfit import SetFitModel
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
# Assuming the text column is named 'text' and the target label column is named 'label'
texts = eval_data['concatenated_text'].values
true_labels = eval_data['class_value'].values

model = SetFitModel.from_pretrained('/content/setfit-finetuned-model')
# Make predictions using your model
predicted_labels = model.predict(texts)

# Evaluate the model performance
accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted')  # Use 'weighted' if multi-class
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')

# Print the evaluation metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Accuracy: 0.7000
F1 Score: 0.6997
Precision: 0.7009
Recall: 0.7000


In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from setfit import SetFitModel
import pandas as pd
import os
import numpy as np

def plot_tsne(embeddings, labels, class_names, output_dir="./tsne_plots"):
    """Generate a t-SNE plot of the embeddings."""
    os.makedirs(output_dir, exist_ok=True)

    # Perform t-SNE dimensionality reduction
    tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=300)
    embeddings_2d = tsne.fit_transform(embeddings)

    # Plot the t-SNE results
    plt.figure(figsize=(10, 8))
    for label in np.unique(labels):
        class_embeddings_2d = embeddings_2d[labels == label]
        plt.scatter(class_embeddings_2d[:, 0], class_embeddings_2d[:, 1], label=class_names[label], alpha=0.7)

    plt.title("t-SNE Visualization of Text Embeddings")
    plt.xlabel("t-SNE Component 1")
    plt.ylabel("t-SNE Component 2")
    plt.legend()
    plt.savefig(f"{output_dir}/tsne_plot.png")
    plt.close()

# Example usage
file_path = "/content/final_dataset.csv"
data = pd.read_csv(file_path)

texts = data["concatenated_text"].tolist()
labels = data["class_value"].tolist()

model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")
embeddings = model.encode(texts)  # Generate embeddings for all text in the dataset

# Define class names for plotting
class_names = {0: "Not Depressed", 1: "Depressed"}

# Generate t-SNE plot
plot_tsne(embeddings, labels, class_names)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_t_sne.py:1162: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(
